In [2]:
# import required libraries to scrape presidential transcripts
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import urllib2
import re

In [35]:
import urllib2,sys,os
from bs4 import BeautifulSoup,NavigableString
from string import punctuation as p
from multiprocessing import Pool
import re, nltk
import requests
reload(sys)


sys.setdefaultencoding('utf8')

#==============================================================================
# Scraping and cleaning one speech from Obama to show the method works
#==============================================================================

obama_4427_url = 'http://www.millercenter.org/president/obama/speeches/speech-4427'
obama_4427 = urllib2.urlopen(obama_4427_url).read()
obama_4427 = BeautifulSoup(obama_4427)

# find the speech itself within the HTML
obama_4427 = obama_4427.find('div',{'id': 'transcript'},{'class': 'displaytext'})

# obama_4427_div.text removes extraneous characters (e.g. '<br/>')
obama_4427 = obama_4427.text.lower()

# for further text analysis, remove punctuation
punctuation = re.compile('[{}]+'.format(re.escape(p)))

# obama_4427_nopunct = [line.decode('utf-8').strip() for line in obama_4427_html.readlines()]

obama_4427 = punctuation.sub('', obama_4427)
obama_4427 = obama_4427.replace('—',' ')
obama_4427 = obama_4427.replace('transcript','')

# divide obama_4427_str_processed into individual words
words = obama_4427.split(' ')

#==============================================================================
# Cleaning links begins below, so that  we can process all 911 speeches through processURL()
#==============================================================================

url = 'http://www.millercenter.org/president/speeches'
url2 = 'http://www.millercenter.org'

conn = urllib2.urlopen(url)
html = conn.read()

miller_center_soup = BeautifulSoup(html)
links = miller_center_soup.find_all('a')

linklist = [tag.get('href') for tag in links if tag.get('href') is not None]

# remove all items in list that don't contain 'speeches'
linkslist = [_ for _ in linklist if re.search('speeches',_)]
del linkslist[0:2]

# concatenate 'http://www.millercenter.org' with end of speech links
every_link_dups = [url2 + end_link for end_link in linkslist]

# remove duplicates
seen = set()
every_link = [] # no duplicates array
for l in every_link_dups:
    if l not in seen:
        every_link.append(l)
        seen.add(l)

# list of presidents (print(len(set(presidents))) = 43 total)
presidents_dups = [l[l.find('president/')+len('president/'):] for l in every_link if 'president' in l]
presidents_dups = [l[0:l.find('/')] for l in presidents_dups]
set2 = set()
presidents = []
for l in presidents_dups:
    if l not in set2:
        presidents.append(l)
        seen.add(l)

presidents = sorted(presidents)

# the following two lines - now commented out - were used to identify duplicates in the original every_link array
# import collections
# print [l for l, count in collections.Counter(every_link).items() if count > 1]

# define a function to clean & store speeches from 'every_link' repository

def processURL(l):
        open_url = urllib2.urlopen(l).read()
        x=urllib2.urlopen(obama_4427_url).read()
        item_soup = BeautifulSoup(x)
        item_div = item_soup.find('div',{'id':'transcript'},{'class':'displaytext'})
        item_str = item_div.text.lower()
        item_str_processed = punctuation.sub('',item_str)
        item_str_processed_final = item_str_processed.replace('—',' ')

        splitlink = l.split("/")
        president = splitlink[4]
        speech_num = splitlink[-1].split("-")[1]
        filename = "{0}_{1}".format(president, speech_num)

        return filename, item_str_processed_final # returning a tuple

# right now, this loop only works for 423 speeches - where are the remaining ones?
for l in every_link[1:423]:
    filename, content = processURL(l) # tuple unpacking
    with open(filename, 'w') as f:
        f.write(content)

In [3]:
import os
from os import path
root = "F:/topicmodel_speeches"
files = os.listdir(root)

In [4]:
#load speeches into a list
docs = list() 
for file in files:
    with open(path.join(root, file), 'r') as fd:
       txt = fd.read()
       docs.append(txt)

In [5]:
import re 
def clean(doc):
    doc = re.sub(r'[^\w\s]*', '', doc) 
    doc = re.sub(r'[\s]+', ' ', doc)
    doc = doc.lower().strip()
    return doc

In [6]:
clean_docs = list()
for doc in docs:
    doc = clean(doc)
    clean_docs.append(doc)

In [5]:
#tokenize speeches
token_docs = list()
for doc in clean_docs:
    token_docs.append(doc.split())

In [6]:
#remove stopwords
stopwords = list()
with open('F:/2060724/topicmodel_stopwords.txt', 'r') as fd:
    for line in fd.readlines():
        stopwords.append(line.strip())

In [7]:
sw_token_docs = list()
for doc in token_docs:
    sw_doc = list()
    for token in doc:
        if not token in stopwords:
            sw_doc.append(token)
    sw_token_docs.append(sw_doc)

In [8]:
# perform topic modelling
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(sw_token_docs)
corpus = [dictionary.doc2bow(doc) for doc in sw_token_docs]

In [7]:
from __future__ import division

import graphlab as gl
import pandas as pd
import pyLDAvis
import pyLDAvis.graphlab

pyLDAvis.enable_notebook()

C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):


In [8]:
doc=gl.SFrame(clean_docs)
sf_paragraphs = doc.rename({'X1': 'paragraph'})
doc.head()

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\pankaj\AppData\Local\Temp\graphlab_server_1471511759.log.0


This non-commercial license of GraphLab Create for academic use is assigned to pankajvshrma@gmail.com and will expire on August 11, 2017.


C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
C:\Users\pankaj\Anaconda2\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


paragraph
fellow citizens of thesenate and the house of ...
whereas it is the duty ofall nations to ...
fellow citizens of thesenate and house of ...
fellow citizens of thesenate and house of ...
i the president of theunited states by my own ...
i meet you upon thepresent occasion with ...
gentlemen of the house ofrepresentatives i have ...
fellowcitizens of thesenate and of the house ...
whereas i have receivedauthentic information ...
fellowcitizens i am againcalled upon by the voice ...


In [9]:
re_words_split = re.compile("(\w+)")
sf_paragraphs['paragraph_words_number'] = sf_paragraphs['paragraph'].apply(lambda p: len(re_words_split.findall(p)) )
sf_paragraphs = sf_paragraphs[sf_paragraphs['paragraph_words_number'] >=25]

In [10]:
docs =  gl.text_analytics.count_ngrams(sf_paragraphs['paragraph'], n=1)

In [11]:
stopwords = gl.text_analytics.stopwords()
# adding some additional stopwords to make the topic model more clear
stopwords |= set(['man', 'mr', 'sir', 'make', 'made', 'll', 'door', 'long', 'day', 'small']) 
docs = docs.dict_trim_by_keys(stopwords, exclude=True)	
docs = docs.dropna()

In [12]:
topic_model = gl.topic_model.create(docs, num_topics=10)

Learning a topic model

Number of documents       622

Vocabulary size     36392

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 952.857ms     | 5.83093e+006   | 0               |

+-----------+---------------+----------------+-----------------+

In [13]:
topic_model.get_topics().print_rows(100)


+-------+------------+------------------+
| topic |    word    |      score       |
+-------+------------+------------------+
|   0   |   people   | 0.0518808217563  |
|   0   |   peace    | 0.0280909494448  |
|   0   |  nations   | 0.0240855117597  |
|   0   |  america   | 0.0215886155404  |
|   0   |    time    | 0.0190136913143  |
|   1   |    law     | 0.0212298873226  |
|   1   |   state    | 0.0205198576463  |
|   1   |    part    | 0.0158104771399  |
|   1   |  question  | 0.0157235347306  |
|   1   |   power    | 0.0139557057405  |
|   2   | government | 0.0582714339429  |
|   2   |  congress  | 0.0272801740345  |
|   2   |  country   | 0.0252658860182  |
|   2   |   great    |  0.022455474643  |
|   2   |  national  | 0.0166140393956  |
|   3   |   states   | 0.0565825654513  |
|   3   |   united   | 0.0503400597837  |
|   3   |   world    | 0.0381020882464  |
|   3   |    war     | 0.0305768759347  |
|   3   |   great    | 0.0113647808662  |
|   4   |    time    |  0.02079749

In [17]:
import pyLDAvis
import pyLDAvis.graphlab
pyLDAvis.enable_notebook()
pyLDAvis.graphlab.prepare(topic_model, docs)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      13.003340        1       1  0.132520 -0.236946
0      12.873568        1       2  0.136203 -0.214329
3      12.633409        1       3 -0.249032 -0.088049
2      11.868215        1       4 -0.238315 -0.055087
6      10.834902        1       5 -0.110124  0.071484
7       8.598419        1       6  0.016187  0.065130
4       8.090620        1       7  0.057047  0.153710
9       7.965658        1       8  0.109521  0.158670
1       7.544927        1       9  0.027125  0.120734
8       6.586941        1      10  0.118869  0.024682, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
20294  Default  5914.000000  government  5914.000000  30.0000  30.0000
25665  Default  5613.000000      states  5613.000000  29.0000  29.0000
9471   Default  6089.000000      people  6089.000000  28.0000  28.0000
17743  Default  5031.000000      united  5031.000000  27.0000  27.0000
10151  Default  3461.000000    congress  3461.000000  26.0000  26.0000
22802  Default  3239.000000   president  3239.000000  25.0000  25.0000
996    Default  3912.000000       world  3912.000000  24.0000  24.0000
27835  Default  3275.000000        time  3275.000000  23.0000  23.0000
3583   Default  3461.000000    american  3461.000000  22.0000  22.0000
8394   Default  3159.000000       peace  3159.000000  21.0000  21.0000
27527  Default  2647.000000     nations  2647.000000  20.0000  20.0000
20907  Default  3040.000000         war  3040.000000  19.0000  19.0000
3065   Default  2399.000000     america  2399.000000  18.0000  18.0000
18513  Default  3535.000000     country  3535.000000  17.0000  17.0000
710    Default  2895.000000       years  2895.000000  16.0000  16.0000
15690  Default  1885.000000    national  1885.000000  15.0000  15.0000
4564   Default  1851.000000         men  1851.000000  14.0000  14.0000
18886  Default  2440.000000      nation  2440.000000  13.0000  13.0000
34361  Default  2132.000000      public  2132.000000  12.0000  12.0000
29025  Default  3378.000000       great  3378.000000  11.0000  11.0000
29621  Default  1888.000000        work  1888.000000  10.0000  10.0000
31546  Default  1815.000000       state  1815.000000   9.0000   9.0000
28328  Default  1845.000000       power  1845.000000   8.0000   8.0000
20350  Default  2547.000000        year  2547.000000   7.0000   7.0000
34242  Default  1566.000000        free  1566.000000   6.0000   6.0000
22598  Default  1563.000000     freedom  1563.000000   5.0000   5.0000
9496   Default  1546.000000    citizens  1546.000000   4.0000   4.0000
32565  Default  1525.000000   americans  1525.000000   3.0000   3.0000
13727  Default  1252.000000      rights  1252.000000   2.0000   2.0000
6249   Default  1576.000000       today  1576.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
11889  Topic10   247.166930  revolution   262.430748   2.6602  -5.3745
6465   Topic10   133.060869    concerns   139.710721   2.6713  -5.9938
33157  Topic10   241.508778    benefits   257.720889   2.6551  -5.3977
4046   Topic10   715.850503        meet   790.762126   2.6206  -4.3111
15690  Topic10  1655.103698    national  1885.743368   2.5896  -3.4729
4564   Topic10  1612.667560         men  1851.105520   2.5822  -3.4989
6237   Topic10   325.438030      entire   355.139931   2.6327  -5.0994
4394   Topic10   158.522552    payments   169.135260   2.6553  -5.8187
24129  Topic10   613.060745      common   752.771266   2.5148  -4.4661
16706  Topic10   414.082408    millions   504.123079   2.5233  -4.8585
33457  Topic10   239.622728       hopes   277.841500   2.5721  -5.4055
21636  Topic10   683.787643     history   925.041694   2.4179  -4.3569
17876  Topic10   369.760219  individual   470.064683   2.4801  -4.9717
12168  Topic10   104.770110  proportion   1

In [49]:
import graphlab as gl
import urllib2
import gensim
import nltk
import re

txt = urllib2.urlopen("https://drive.google.com/open?id=0B5g0WZIzJLuEU3hqczNEb21MaGM").read()

re_words_split = re.compile("(\w+)")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def txt2words(s):
    s = re.sub("[^a-zA-Z]", " ", s).lower()
    return re_words_split.findall(s)

class MySentences(object):
        def __init__(self, txt):
            self._txt = txt.decode("utf8") 
            
        def __iter__(self):
            """
            Split the English text into sentences and then to words using NLTK
            :param txt: input text.    
            :param remove_none_english_chars: if True then remove none English chars from text
            :return: list of words in which each list consists of single sentence's words from the original input text.
            :rtype: str
            """                
            # split text into sentences using NLTK package
            for s in tokenizer.tokenize(self._txt):                                    
                yield txt2words(s)

sentences = MySentences(txt)
model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=3, workers=4)


In [55]:
print model.most_similar("obama")


[(u'com', 0.7005459666252136), (u'google', 0.685105562210083), (u'a', 0.6578261256217957), (u'true', 0.6530385613441467), (u'link', 0.6439136862754822), (u'd', 0.6416248083114624), (u'text', 0.6234346628189087), (u'file', 0.6168403029441833), (u'docs', 0.6084883809089661), (u'hqczneb', 0.597303569316864)]


In [60]:
import numpy as np
def txt2avg_vector(txt, w2v_model):
    words = [w for w in txt2words(txt.lower()) if w in w2v_model]
    v = np.mean([w2v_model[w] for w in words],axis=0)    
    return v

sf_paragraphs['mean_vector'] = sf_paragraphs['paragraph'].apply(lambda p: txt2avg_vector(p, model))

In [61]:
#construncting nearest neighbors model
nn_model = gl.nearest_neighbors.create(sf_paragraphs, features=['mean_vector'])

#calaculating the two nearest neighbors of each paragraph from all the paragraphs 
r = nn_model.query(sf_paragraphs, k=2)
r.head(10)

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 17.634ms     |

...

+--------------+-------------+--------------+

| 1            | 0           | 5.003ms      |

| Done         |             | 638.43ms     |

+--------------+-------------+--------------+

query_label,reference_label,distance,rank
0,0,0.0,1
0,247,0.000894301737532,2
1,1,0.0,1
1,75,0.00177082049266,2
2,2,0.0,1
2,16,0.000928484636116,2
3,3,0.0,1
3,110,0.00074604515377,2
4,4,0.0,1
4,139,0.00162629810938,2


In [62]:
#filter out paragraphs that are exactly exactly the same
r = r[r['distance'] != 0]

#filter out paragraphs that are with distance >= 0.1
r = r[r['distance'] < 0.08]
r

query_label,reference_label,distance,rank
0,247,0.000894301737532,2
1,75,0.00177082049266,2
2,16,0.000928484636116,2
3,110,0.00074604515377,2
4,139,0.00162629810938,2
5,11,0.000744305314392,2
6,93,0.00553016265452,2
7,103,0.000770922708859,2
8,214,0.00120041979798,2
9,371,0.00371562996025,2


In [63]:
sf_paragraphs = sf_paragraphs.add_row_number('query_label')
sf_paragraphs = sf_paragraphs.add_row_number('reference_label')
sf_similar = r.join(sf_paragraphs, on="query_label").join(sf_paragraphs, on="reference_label")

In [65]:
sf_similar[['paragraph', 'paragraph.1', 'distance']]

paragraph,paragraph.1,distance
fellow citizens of thesenate and the house of ...,senator wagner governorlehman ladies and ...,0.000894301737532
whereas it is the duty ofall nations to ...,to the house ofrepresentatives of the ...,0.00177082049266
fellow citizens of thesenate and house of ...,i trust i do not deceivemyself when i indulge ...,0.000928484636116
fellow citizens of thesenate and house of ...,fellowcitizens of thesenate and house of ...,0.00074604515377
i the president of theunited states by my own ...,by the president of theunited states of america ...,0.00162629810938
i meet you upon thepresent occasion with ...,fellow citizens of thesenate and of the house ...,0.000744305314392
gentlemen of the house ofrepresentatives i have ...,whereas by an act of thecongress of the united ...,0.00553016265452
fellowcitizens of thesenate and of the house ...,fellow citizens of thesenate and house of ...,0.000770922708859
whereas i have receivedauthentic information ...,gentlemen of the congressin fulfilling at this ...,0.00120041979798
fellowcitizens i am againcalled upon by the voice ...,dr newhouse chancellortolley governor and mrs ...,0.00371562996025


In [66]:
print sf_similar[1]['paragraph']
print "-"*100
print sf_similar[1]['paragraph.1']

whereas it is the duty of all nations to acknowledge the providence of almighty god to obey his will to be grateful for his benefits and humbly to implore his protection and favor and whereas both houses of congress have by their joint committee requested me to recommend to the people of the united states a day of public thanksgiving and prayer to be observed by acknowledging with grateful hearts the many signal favors of almighty god especially by affording them an opportunity peaceably to establish a form of government for their safety and happiness now therefore i do recommend and assign thursday the 26th day of november next to be devoted by the people of these states to the service of that great and glorious being who is the beneficent author of all the good that was that is or that will be that we may then all unite in rendering unto him our sincere and humble thanks for his kind care and protection of the people of this country previous to their becoming a nation for the signal 

In [12]:
import logging, gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=25, update_every=1, passes=20)

In [13]:
#print 25 topics out to 20 words
t=0
for i in lda.show_topics(num_topics=25, num_words=20, log=False, formatted=True):
    print "Topic # ", t , i
    t = t + 1

Topic #  0 (0, u'0.007*people + 0.006*kosovo + 0.006*war + 0.006*bosnia + 0.005*army + 0.005*peace + 0.005*union + 0.005*military + 0.005*men + 0.004*nato + 0.004*time + 0.004*compromise + 0.004*promise + 0.004*allies + 0.004*drought + 0.003*mccain + 0.003*europe + 0.003*lives + 0.003*berlin + 0.003*issue')
Topic #  1 (1, u'0.007*increase + 0.007*people + 0.007*states + 0.006*government + 0.006*year + 0.006*work + 0.006*united + 0.005*wages + 0.005*secretary + 0.004*1892 + 0.004*union + 0.004*general + 0.004*american + 0.004*great + 0.004*cent + 0.004*transportation + 0.004*products + 0.004*trade + 0.004*president + 0.003*1890')
Topic #  2 (2, u'0.018*business + 0.011*law + 0.010*labor + 0.009*government + 0.009*great + 0.008*public + 0.008*national + 0.007*conditions + 0.007*power + 0.007*men + 0.006*industrial + 0.006*corporations + 0.006*interstate + 0.006*legislation + 0.005*commerce + 0.005*country + 0.005*man + 0.005*work + 0.005*congress + 0.004*control')
Topic #  3 (3, u'0.010*

In [14]:
#print topic weight in each speech    
count=1
for doc in sw_token_docs:
    vec = dictionary.doc2bow(doc)
    print "Speech # ", count, lda[vec]
    #print first 100 words of speech to verify correct speech
    print doc[0:100]
    count = count + 1


Speech #  1 [(9, 0.50640713197693921), (18, 0.1351239923731434), (20, 0.35489200038374513)]
['fellow', 'citizens', 'senate', 'house', 'representatives', 'vicissitudes', 'incident', 'life', 'event', 'filled', 'greater', 'anxieties', 'notification', 'transmitted', 'order', 'received', 'fourteenth', 'day', 'present', 'month', 'hand', 'summoned', 'country', 'voice', 'hear', 'veneration', 'love', 'retreat', 'chosen', 'fondest', 'predilection', 'flattering', 'hopes', 'immutable', 'decision', 'asylum', 'declining', 'years', 'retreat', 'rendered', 'day', 'dear', 'addition', 'habit', 'inclination', 'frequent', 'interruptions', 'health', 'gradual', 'waste', 'committed', 'time', 'hand', 'magnitude', 'difficulty', 'trust', 'voice', 'country', 'called', 'sufficient', 'awaken', 'wisest', 'experienced', 'citizens', 'distrustful', 'scrutiny', 'qualification', 'overwhelm', 'dispondence', 'inheriting', 'inferior', 'endowments', 'nature', 'unpractised', 'duties', 'civil', 'administration', 'peculiarly', 

ValueError: I/O operation on closed file

In [15]:
lda.print_topics(20)

[(3,
  u'0.010*america + 0.009*people + 0.007*iraq + 0.006*american + 0.006*world + 0.005*nation + 0.005*country + 0.005*men + 0.004*united + 0.004*security'),
 (24,
  u'0.014*care + 0.014*health + 0.009*insurance + 0.008*people + 0.008*system + 0.006*government + 0.006*americans + 0.005*plan + 0.005*american + 0.004*inflation'),
 (13,
  u'0.012*vietnam + 0.008*energy + 0.007*congress + 0.007*president + 0.006*south + 0.005*american + 0.005*peace + 0.005*government + 0.005*talks + 0.004*states'),
 (11,
  u'0.011*war + 0.009*people + 0.008*government + 0.008*peace + 0.006*world + 0.005*germany + 0.005*american + 0.005*time + 0.005*great + 0.005*united'),
 (1,
  u'0.007*increase + 0.007*people + 0.007*states + 0.006*government + 0.006*year + 0.006*work + 0.006*united + 0.005*wages + 0.005*secretary + 0.004*1892'),
 (19,
  u'0.023*united + 0.017*states + 0.014*nations + 0.011*world + 0.008*peace + 0.007*american + 0.006*war + 0.005*seas + 0.004*attack + 0.004*policy'),
 (15,
  u'0.024*pre

In [ ]:
!